# PDF Table Parsing

In [1]:
# !pip install layoutparser # Install the base layoutparser library with  
# !pip install "layoutparser[layoutmodels]" # Install DL layout model toolkit 
# !pip install "layoutparser[ocr]" # Install OCR toolkit

In [2]:
# install detectron2 - option 1 (not working on mac)

# !pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2' 

In [3]:
# install detectron2 - option 2 (working on mac)

# !git clone https://github.com/facebookresearch/detectron2.git
# !CC=clang CXX=clang++ ARCHFLAGS="-arch x86_64" python -m pip install -e detectron2

In [4]:
# install poppler - (required for pdf -> image step)

# !brew install poppler

In [5]:
data_dir = '../data/Zoning Regulation Tables/'

# 0. Demo

In [ ]:
pdf_fp = data_dir + 'tables/1.pdf' # demo file

## 0.1 Layout Parser

In [ ]:
import layoutparser as lp
import detectron2

In [ ]:
pdf_layout, pdf_images = lp.load_pdf(pdf_fp, load_images=True)

In [ ]:
model = lp.models.Detectron2LayoutModel(
#             config_path ='lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config', # In model catalog
            config_path ='lp://TableBank/faster_rcnn_R_50_FPN_3x/config', # In model catalog
#             label_map   ={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}, # In model`label_map`
#             extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8] # Optional
        )

# model = lp.AutoModel("lp://detectron2/TableBank/faster_rcnn_R_50_FPN_3x")
res = model.detect(pdf_images[0])

In [ ]:
lp.draw_box(pdf_images[0], res)

In [ ]:
res

## 0.2 PDF elements

In [ ]:
from pdfminer.high_level import extract_pages

# pdf_fp = list_of_table_files[0]
for page_layout in extract_pages(pdf_fp):
    for element in page_layout:
        print(element)

# 1. Data Info

In [27]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np

import layoutparser as lp
import detectron2

from pdfminer.high_level import extract_pages

In [28]:
# debug = True

In [29]:
# pdf_filepath = data_dir + 'tables/2.pdf' # test file

In [30]:
# page = list(extract_pages(pdf_filepath))[0]

In [44]:
LARGE_NUM = 1e10

def handle_multi_candidate_orientation(zone_loc_dict, zone_codes, axis):
    
    value_lst = [[i[axis] for i in zone_loc_dict[z]] for z in zone_codes]
    #value_lst = [[0], [0.1, -0.2], [0.03], [0.15, 0.6, -0.05]]
    count_dict = {i: len(value_lst[i]) for i in range(len(value_lst))}

    # x-axis
    fixed_nums = []
    dynamic_nums = []
    dynamic_lst_idx_2_old_idx = {}
    counter = 0 
    for i in range(len(value_lst)):
        l = value_lst[i]
        if len(l) == 0:
            continue
        if len(l) == 1:
            fixed_nums.append(l[0])
        else:
            dynamic_nums.append(l)
            dynamic_lst_idx_2_old_idx[counter] = i
            counter += 1
    dynamic_combs = np.array(eval('np.meshgrid(' + ','.join([str(i) for i in dynamic_nums]) + ')'))\
                      .T.reshape(-1, len(dynamic_nums))

    min_std = LARGE_NUM
    best_nums = []
    for comb in dynamic_combs:
        all_nums = fixed_nums + list(comb)
        std = np.std(all_nums)
        if std < min_std:
            min_std = std
            best_nums = all_nums
    #min_std, all_nums

    fixed_count = len(fixed_nums)
    selected_idx = {i: 0 for i in range(len(value_lst))}
    for i in range(len(value_lst)):
        if len(value_lst[i]) == 0:
            selected_idx[i] = -1
    for idx in range(len(dynamic_nums)):
        selected_num = best_nums[fixed_count + idx]
        curr_zone_idx = dynamic_lst_idx_2_old_idx[idx]
        selected_idx[curr_zone_idx] = (value_lst[curr_zone_idx].index(selected_num))
    return selected_idx, min_std

In [46]:
%run ../src/utils.py
glove_dict = load_glove_embeddings()

In [47]:
np.array(glove_dict['is']).shape

(300,)

In [48]:
from nltk.tokenize import word_tokenize
import string 
import re
printable_char = set(list(string.printable))

def tokenizer(text):
    text = ''.join([s for s in text if s in printable_char]).lower()
    text = re.sub('\t+', ' ', text)
    text = re.sub('\n+', ' ', text)
    text = re.sub(' +', ' ', text).strip()
    tokens = word_tokenize(text)
    return tokens

def glove_embedding(text):
    tokens = tokenizer(text)
    embedding = np.zeros(300)
    counter = 0
    for t in tokens:
        try:
            embedding += np.array(glove_dict[t])
            counter += 1
        except:
            continue
    if counter == 0:
        return embedding
    return embedding / counter

from numpy.linalg import norm
def cos_similarity(v1, v2):
    norms = (norm(v1)*norm(v2))
    if norms != 0:
        return np.dot(v1, v2) / (norm(v1)*norm(v2))
    else:
        return 0

def clean_num(text):
    text = re.sub('[^0-9.,]', ' ', text)
    tokens = word_tokenize(text)
    return [t for t in tokens if len(t.strip()) > 0]

In [49]:
# tokenizer('Minimun lot size').shape
data_dir

'../data/Zoning Regulation Tables/'

In [50]:
class DocBox:
    def __init__(self, box, page_dim):
        if isinstance(box, dict) and page_dim == None:
            self.x0 = box['x0']
            self.x1 = box['x1']
            self.y0 = box['y0']
            self.y1 = box['y1']
            self.text = ''
        else:
            self.x0 = round(box.x0, 2)
            self.x1 = round(box.x1, 2)
            self.y0 = round(page_dim[1] - box.y0, 2)
            self.y1 = round(page_dim[1] - box.y1, 2)
            try:
                self.text = box.get_text()
            except:
                self.text = ''

    def __repr__(self):
        return "<docbox x0: {}, y0: {}, x1: {}, y1: {}, text: '{}'"\
                    .format(self.x0, self.y0, self.x1, self.y1, self.text.strip())
    def get_text(self):
        return self.text
    
    def overlap(self, other_box, scaling = 0.1):
        if self.x1 < other_box.x0 * (1-scaling) or self.x0 > other_box.x1 * (1+scaling):
            return False
        if self.y1 < other_box.y0 * (1-scaling) or self.y0 > other_box.y1 * (1+scaling):
            return False
        return True
class Table:
    def __init__(self, loc, page_num, zone_codes, debug = False, multi_page = False, scaling = 0.1):
        x1, y1, x2, y2 = loc
        self.x1 = x1 * (1 - scaling)
        self.y1 = y1 * (1 - scaling)
        self.x2 = x2 * (1 + scaling)
        self.y2 = y2 * (1 + scaling)
        self.page_num = page_num
        self.multi_page = multi_page
        self.zone_codes = zone_codes
        self.debug = debug
        self.zone_loc_dict = {c: [] for c in zone_codes}
        
    
    def attach_zones_to_table(self, page_text_dict):
        if self.page_num not in page_text_dict:
            return 
        if len(page_text_dict[self.page_num]) == 0:
            return
        zone_boxes = page_text_dict[self.page_num]
        for box in zone_boxes:
            box_x1 = box[0].x0
            box_y1 = box[0].y0
            box_x2 = box[0].x1
            box_y2 = box[0].y1
            if box_x1 < self.x1 or box_y1 < self.y1 or box_x2 > self.x2 or box_y2 > self.y2:
                # the box is out of the table
                continue
            box_info = {
                'x1': box_x1,
                'y1': box_y1,
                'x2': box_x2,
                'y2': box_y2,
                'text': box[0].get_text()
            }
            self.zone_loc_dict[box[1]].append(box_info)
    
    def find_table_orientation(self):
        
        # check combination options
        if sum([len(self.zone_loc_dict[z]) > 1 for z in self.zone_codes]) == 0:
            if self.debug:
                print(['DEBUG: all zones have only ONE text found in this table!'])
            x1_lst = [self.zone_loc_dict[z][0]['x1'] for z in self.zone_codes if len(self.zone_loc_dict[z]) > 0]
            x2_lst = [self.zone_loc_dict[z][0]['x2'] for z in self.zone_codes if len(self.zone_loc_dict[z]) > 0]
            y1_lst = [self.zone_loc_dict[z][0]['y1'] for z in self.zone_codes if len(self.zone_loc_dict[z]) > 0]
            y2_lst = [self.zone_loc_dict[z][0]['y2'] for z in self.zone_codes if len(self.zone_loc_dict[z]) > 0]
            std_lst = [np.std(x1_lst), np.std(x2_lst), np.std(y1_lst), np.std(y2_lst)]
            min_loc = np.argmin(std_lst)
            self.min_std = min(std_lst)
            if min_loc == 0 or min_loc == 1:
                self.orientation = 'vertical'
            else:
                self.orientation = 'horizontal'
            self.selected_index = {z: 0 for z in self.zone_codes}
            for z in self.zone_codes:
                if len(self.zone_loc_dict[z]) == 0:
                    self.selected_index[z] = -1
        else:
            if self.debug:
                print(['DEBUG: some zones have MORE THAN ONE text found in this table!'])
            
            # x1-axis
            final_idx_lst_x1 = []
            x1_idx, x1_std = handle_multi_candidate_orientation(self.zone_loc_dict, self.zone_codes, axis = 'x1')
            y1_idx, y1_std = handle_multi_candidate_orientation(self.zone_loc_dict, self.zone_codes, axis = 'y1')
            x2_idx, x2_std = handle_multi_candidate_orientation(self.zone_loc_dict, self.zone_codes, axis = 'x2')
            y2_idx, y2_std = handle_multi_candidate_orientation(self.zone_loc_dict, self.zone_codes, axis = 'y2')
            min_std_idx = np.argmin([x1_std, x2_std, y1_std, y2_std])
            if min_std_idx == 0 or min_std_idx == 1:
                self.orientation = 'vertical'
            else:
                self.orientation = 'horizontal'
            self.min_std = [x1_std, x2_std, y1_std, y2_std][min_std_idx]
            selected_index = [x1_idx, x2_idx, y1_idx, y2_idx][min_std_idx]
            self.selected_index = {zone_codes[i]: selected_index[i] for i in range(len(self.zone_codes))}
    
    def box_max_sim_score(self, box):
        try:
            box_text_vec = tokenizer(box.get_text())
            best_score = max([cos_similarity(vec, box_text_vec) for vec in all_variable_vec])
            return best_score
        except:
            return 0

    def find_variable_box(self):
        zone_boxes = list(self.pdfminer_results)
        if len(zone_boxes) == 0:
            self.variable_box = None
            self.max_sim_score = 0
            return
        else:
            sim_scores = [self.box_max_sim_score(box) for box in zone_boxes]
            bext_box_idx = np.argmax(sim_scores)
            self.variable_box = DocBox(zone_boxes[bext_box_idx], self.page_dim)
            self.max_sim_score = max(sim_scores)
        #return zone_boxes[bext_box_idx]
        
    def find_variable_values(self):
        raw_results = {}
        if self.orientation == 'vertical':
            x0, x1 = self.variable_box.x0, self.variable_box.x1
            for zone_idx in range(len(self.zone_codes)):
                zone = self.zone_codes[zone_idx]
                raw_results[zone] = []
                if len(self.zone_loc_dict[zone]) == 0:
                    continue
                selected_text_box = self.zone_loc_dict[zone][self.selected_index[zone]]
                y0, y1 = selected_text_box['y1'], selected_text_box['y2']
                target_box = DocBox({'x0': x0, 'x1': x1, 'y0': y0, 'y1': y1}, None)
                #print(zone, target_box)
                for box in self.pdfminer_results:
                    tmp_box = DocBox(box, self.page_dim)
                    if target_box.overlap(tmp_box) and tmp_box.get_text() != '':
                        raw_results[zone].append(tmp_box.get_text())
        elif self.orientation == 'horizontal':
            y0, y1 = self.variable_box.y0, self.variable_box.y1
            for zone_idx in range(len(self.zone_codes)):
                zone = self.zone_codes[zone_idx]
                raw_results[zone] = []
                if len(self.zone_loc_dict[zone]) == 0:
                    continue
                selected_text_box = self.zone_loc_dict[zone][self.selected_index[zone]]
                x0, x1 = selected_text_box['x1'], selected_text_box['x2']
                target_box = DocBox({'x0': x0, 'x1': x1, 'y0': y0, 'y1': y1}, None)
                #print(zone, target_box)
                for box in self.pdfminer_results:
                    tmp_box = DocBox(box, self.page_dim)
                    if target_box.overlap(tmp_box) and tmp_box.get_text() != '':
                        raw_results[zone].append(tmp_box.get_text())
        
        cleaned_results = self.clean_final_results(raw_results)
        self.raw_results = raw_results
        self.cleaned_results = cleaned_results
        #return final_results
    
    def clean_final_results(self, raw_results):
        '''
        TODO: improve cleaning procedure
        '''
        final_result = {}
        
        for zone in raw_results.keys():
            preferred_results = []
            all_results = []
            zone_final = None
            for text in raw_results[zone]:
                tokens = clean_num(text)
                if 'sq ft' in text.lower() or 'sq' in text.lower():
                    preferred_results += tokens
                all_results += tokens
            if len(preferred_results) > 0 and len(preferred_results) < len(all_results):
                zone_final = preferred_results
            else:
                zone_final = all_results
            zone_final = [re.sub('[,.]', '', num) for num in zone_final]
            zone_final = [float(num) for num in zone_final if len(num) > 0]
            final_result[zone] = zone_final
        return final_result
    
    def eval_table(self):
        # number of zones found
        found_hit = sum([len(self.zone_loc_dict[c]) > 0 for c in self.zone_codes])
        found_count = sum([len(self.zone_loc_dict[c]) for c in self.zone_codes])
        found_hit_score = found_hit / len(self.zone_codes)
        found_count_score = found_count / len(self.zone_codes)
        
        # zone location distribution
        #min_std_score = -self.min_std
        #if np.isnan(min_std_score):
        #    min_std_score = LARGE_NUM
        
        # confidence of variable score
        variable_confidence_score = self.max_sim_score
        
        return [found_hit_score, found_count_score, variable_confidence_score]

class Document:
    def __init__(self, pdf_filepath, zone_codes):
        self.zone_codes = zone_codes
        self.pdf_filepath = pdf_filepath
        self.table_lst = []
        self.initial_detection()
    
    def initial_detection(self):
        #### ocr level extractions ####
        pdf_layout, pdf_images = lp.load_pdf(self.pdf_filepath, load_images = True)
        model = lp.models.Detectron2LayoutModel(config_path ='lp://TableBank/faster_rcnn_R_50_FPN_3x/config')
        
        ocr_results = []
        for page in pdf_images:
            res = model.detect(page)
            ocr_results.append(res)
        self.ocr_results = ocr_results
        
        self.page_table_dict = {}
        for page_i in range(len(ocr_results)):
            res = ocr_results[page_i]
            self.page_table_dict[page_i] = []
            for table_loc in ocr_results[page_i]._blocks:
                x_1 = round(table_loc.block.x_1, 2)
                y_1 = round(table_loc.block.y_1, 2)
                x_2 = round(table_loc.block.x_2, 2)
                y_2 = round(table_loc.block.y_2, 2)
                self.page_table_dict[page_i].append((x_1, y_1, x_2, y_2))
                
        #### text level extractions ####
        pdfminer_results = list(extract_pages(self.pdf_filepath)) # pdfminer results
        self.pdfminer_results = pdfminer_results
        
        self.page_text_dict = {}
        self.size_dict = {}
        for page_i in range(len(pdfminer_results)):
            res = pdfminer_results[page_i]
            self.page_text_dict[page_i] = []
            page_dim = (res.x1, res.y1)
            self.size_dict[page_i] = page_dim
            for box in res:
                for z in self.zone_codes:
                    try:
                        text = box.get_text()
                    except:
                        continue
                    if z in text:
                        self.page_text_dict[page_i].append((DocBox(box, page_dim), z))

    def parse_tables(self):
        for page_num in self.page_table_dict.keys():
            page_lst = self.page_table_dict[page_num]
            for table_loc in page_lst:
                # setup table
                t = Table(table_loc, page_num, self.zone_codes)
                t.pdfminer_results = self.pdfminer_results[page_num]
                t.page_dim = self.size_dict[page_num]
                t.attach_zones_to_table(self.page_text_dict)
                self.table_lst.append(t)
                
                # parse table
                t.find_table_orientation()
                t.find_variable_box()
    
    def inference(self):
        scores = []
        for t in self.table_lst:
            #inference
            t.find_variable_values()
            # evaluate table
            table_score = sum(t.eval_table())
            scores.append(table_score)
        #scores = np.array(scores)
        #scores = ((scores - scores.mean(axis = 0)) / scores.std(axis = 0)).sum(axis = 1)
        best_table_idx = np.argmax(scores)
        self.best_table = self.table_lst[best_table_idx]

## 1.1 Test

In [51]:
zone_codes = ['R4', 'WR', 'R6', 'R8-12', 'MFR']
pdf_filepath = data_dir + 'tables/2.pdf' # test file
doc = Document(pdf_filepath, zone_codes)

The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


In [52]:
# doc.page_table_dict

In [53]:
# doc.page_text_dict

In [54]:
doc.parse_tables()
doc.inference()
doc.best_table.raw_results

/opt/anaconda3/envs/dsml/lib/python3.8/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/envs/dsml/lib/python3.8/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/envs/dsml/lib/python3.8/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'R4': ['R4\n',
  '8,000 sq ft\n',
  '60 feet\n',
  '25 feet\n',
  '15 total\n',
  'internal\n',
  '(no less\n'],
 'WR': ['- not\n',
  '(no less\n',
  'WR\n',
  '9,600 sq ft\n',
  'variable\n',
  '25 feet\n',
  '15 total\n'],
 'R6': ['feet\n',
  'internal\n',
  '55 feet\n',
  'side)\n',
  '(no less\n',
  'than 5\n',
  'R6\n',
  '6,000 sq ft\n',
  '50 feet\n',
  '20 feet\n',
  '10 total\n'],
 'R8-12': ['than 5\n',
  'R8-123 Detached 4,000\nsq ft\n',
  '45 feet\n',
  '15 feet\n',
  '10 total\n',
  'internal\n',
  '(25 feet\n',
  '(no less\n',
  '50 feet\n',
  'max.)8\n'],
 'MFR': ['or\n',
  'buildings\n',
  'on site\n',
  'MFR\n',
  'None\n',
  '20 feet4 Variable 10 feet\n',
  'between\n',
  'The Lake Stevens Municipal Code is current through Ordinance 1126, and legislation passed through October 13, 2021.\n']}

In [55]:
doc.best_table.cleaned_results

{'R4': [8000.0],
 'WR': [9600.0],
 'R6': [6000.0],
 'R8-12': [8.0, 123.0, 4000.0],
 'MFR': [20.0, 4.0, 10.0, 1126.0, 13.0, 2021.0]}

## 1.2 Production

In [56]:
meta_data_df = pd.read_csv(data_dir + 'tables/tables_meta_data.csv')
meta_data_df.head()

,file,zones,type,note
0,1,"R-1, R-4, R-6, R-8, R-18, MR",single_table,NaN
1,2,"R4, WR, R6, R8-12, MFR",single_table,multi-page
2,3,"MF-10, MF-18, MF-C",single_table,multi-page
3,4,"R-6, R-7.5, R-10, R-12, R-15",single_table,multi-page
4,5,"R-1, R-2, R-3, R-4, R-PD, R-MH",single_table,NaN


In [57]:
raw_results = []
cleaned_results = []
for i in tqdm(range(meta_data_df.shape[0])):
    file_data = meta_data_df.iloc[i]
    pdf_filepath = data_dir + 'tables/{}.pdf'.format(file_data['file'])
    zone_codes = [zone.strip() for zone in file_data['zones'].split(',')]
    
    doc = Document(pdf_filepath, zone_codes)
    doc.parse_tables()
    doc.inference()
    raw_results.append(doc.best_table.raw_results)
    cleaned_results.append(doc.best_table.cleaned_results)

  0%|                                          | 0/39 [00:00<?, ?it/s]The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
/opt/anaconda3/envs/dsml/lib/python3.8/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/envs/dsml/lib/python3.8/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/anaconda3/envs/dsml/lib/python3.8/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  3%|▊                                 | 1/39 [00:06<03:49,  6.04s/it]The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std
/opt/anaconda3/envs/dsml/lib/python3.8/site-packages/numpy/core/_methods.py:262: Runtim

KeyboardInterrupt: 

In [58]:
cleaned_results

[{'R-1': [], 'R-4': [], 'R-6': [], 'R-8': [], 'R-18': [], 'MR': []},
 {'R4': [8000.0],
  'WR': [9600.0],
  'R6': [6000.0],
  'R8-12': [8.0, 123.0, 4000.0],
  'MFR': [20.0, 4.0, 10.0, 1126.0, 13.0, 2021.0]},
 {'MF-10': [], 'MF-18': [], 'MF-C': []},
 {'R-6': [], 'R-7.5': [], 'R-10': [], 'R-12': [], 'R-15': []},
 {'R-1': [], 'R-2': [2.0], 'R-3': [3.0], 'R-4': [4.0], 'R-PD': [], 'R-MH': []},
 {'SF-1': [],
  'SF-2': [],
  'SF-3': [],
  'SF-4': [],
  'SF-A': [],
  'MX-D': [],
  'MF-1': [],
  'MF-2': [],
  'MF-3': [],
  'MX-R': []},
 {'RE': [],
  'R-40': [],
  'R-30': [],
  'R-20': [],
  'R-15': [],
  'R-D': [],
  'R-MF': [],
  'MH': [],
  'CC2': []},
 {'SF-1': [],
  'SF-2': [],
  'SF-3': [],
  'SF-4': [],
  'SF-5': [],
  'SF-6': [],
  'TF-1': [],
  'MF1': [],
  'MF2': [],
  'MH1': [],
  'MH2': []},
 {'FA': [], 'RS': [], 'RG': [], 'CO': [], 'CN': [], 'CG': [], 'LM': []},
 {'OS-P': [],
  'OS-A': [],
  'RE': [],
  'AR': [],
  'R-40': [],
  'R-25': [],
  'R-10': [],
  'B': [],
  'WB': []},
 {'R-

In [59]:
raw_results

[{'R-1': [], 'R-4': [], 'R-6': [], 'R-8': [], 'R-18': [], 'MR': []},
 {'R4': ['R4\n',
   '8,000 sq ft\n',
   '60 feet\n',
   '25 feet\n',
   '15 total\n',
   'internal\n',
   '(no less\n'],
  'WR': ['- not\n',
   '(no less\n',
   'WR\n',
   '9,600 sq ft\n',
   'variable\n',
   '25 feet\n',
   '15 total\n'],
  'R6': ['feet\n',
   'internal\n',
   '55 feet\n',
   'side)\n',
   '(no less\n',
   'than 5\n',
   'R6\n',
   '6,000 sq ft\n',
   '50 feet\n',
   '20 feet\n',
   '10 total\n'],
  'R8-12': ['than 5\n',
   'R8-123 Detached 4,000\nsq ft\n',
   '45 feet\n',
   '15 feet\n',
   '10 total\n',
   'internal\n',
   '(25 feet\n',
   '(no less\n',
   '50 feet\n',
   'max.)8\n'],
  'MFR': ['or\n',
   'buildings\n',
   'on site\n',
   'MFR\n',
   'None\n',
   '20 feet4 Variable 10 feet\n',
   'between\n',
   'The Lake Stevens Municipal Code is current through Ordinance 1126, and legislation passed through October 13, 2021.\n']},
 {'MF-10': [], 'MF-18': [], 'MF-C': []},
 {'R-6': [], 'R-7.5': [],